In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

import talib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import mplfinance as mpf
import requests
import csv

import warnings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [2]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [3]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# # Define the ticker symbols as a list; eg. TSM,MSFT,AMZN
# ticker_symbols = input("Enter stock tickers separated by commas:") 
# ticker_symbols = ticker_symbols.split(',')
ticker_symbols = [
    'GOOGL'
]

# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]


ticker_dict = {}

In [30]:
# Daily quote section
for symbol in ticker_symbols:

    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':

            # selected_cols = [
            #     '4. close'
            # ]

            df_stock_base = pd.DataFrame(value).transpose() # tranpose the dataframe and sub select selected cols

            # Rename columns
            df_stock_base.rename(
                columns={
                    '1. open': 'open'
                    ,'2. high': 'high'
                    ,'3. low': 'low'
                    ,'4. close': 'close'
                    ,'5. volume': 'volume'
                    }
                ,inplace=True
                )
            
            for col in df_stock_base.columns:
                if col == 'volume':
                    df_stock_base[col] = df_stock_base[col].astype(str).apply(lambda x: int(x))
                else:
                    df_stock_base[col] = df_stock_base[col].astype(str).apply(lambda x: float(x))
                    df_stock_base[col] = df_stock_base[col].round(2)


            df_stock_base['symbol'] = symbol
            # convert string index to datetime index
            df_stock_base.index = pd.to_datetime(df_stock_base.index)


            # 锤子线
            
            # 分为当日是阴线， 阳线
            # 1. 出现在明显的下行趋势中
            # 过去一段period里的斜率是负数
            # max() high 与 min() high 之间的差值有超过10% 或者 15%，可以调参


            # 2. 下影线是实体部分的两倍以上



            # 3. 成交量不能是缩量， 至少要和平均成交量持平， 越大越好
            # 当日成交量>=，过去一段时间成交量的均值, 或者前75%


In [31]:
# 锤子线

# 分为当日是阴线， 阳线
# 1. 出现在明显的下行趋势中
# 过去一段period里的斜率是负数


# 2. 下影线是实体部分的两倍以上

# 3. 成交量不能是缩量， 至少要和平均成交量持平， 越大越好
# 当日成交量>=，过去一段时间成交量的均值, 或者前75%

def K_hammer(window):
    # in pervious window days
    window = window
    # 阴线
    for index, row in df_stock_base.iterrows():
        if row['open'] > row['close']:

            print(index, "open > close")
            # print(index-timedelta(1))
            print(df_stock_base[index])
            # print(df_stock_base['high'].max())
            # if (max(row['high']) - min(row['high'])) / max(row['high']) >= 0.15 \
            #       and abs(row['close'] - row['low']) >= 2 * abs(row['open'] - row['close']) \
            #       and min(row['high'].index) > max(row['high'].index) \
            #       and  


        # 阳线
        if row['open'] < row['close']:
            print(index, "open < close")
        # print(df_stock_base[index])

    return

In [32]:
fig = plt.figure(figsize=(12,8))

plt.rcParams['font.sans-serif'] = ['SimHei']

df = df_stock_base

df['hammer_head'] = talib.CDLHAMMER(
    df['open'].values
    ,df['high'].values
    ,df['low'].values
    ,df['close'].values
)

df['Inverted_hammer_head'] = talib.CDLINVERTEDHAMMER(
    df['open'].values
    ,df['high'].values
    ,df['low'].values
    ,df['close'].values
)


pattern = df[(df['hammer_head'] == 100) | (df['hammer_head'] == -100)]

df

,open,high,low,close,volume,symbol,hammer_head,Inverted_hammer_head
2024-05-20,176.19,178.77,176.08,176.92,22554400,GOOGL,0,0
2024-05-17,174.18,176.26,173.69,176.06,24479262,GOOGL,0,0
2024-05-16,173.29,175.12,172.69,174.18,27867947,GOOGL,0,0
2024-05-15,170.63,172.65,170.51,172.51,26948370,GOOGL,0,0
2024-05-14,169.77,171.25,168.80,170.34,25127138,GOOGL,0,0
2024-05-13,164.26,169.28,164.00,169.14,31327602,GOOGL,0,0
2024-05-10,168.02,169.85,166.19,168.65,29799931,GOOGL,0,0
2024-05-09,169.39,170.69,168.18,169.96,15346700,GOOGL,0,0
2024-05-08,169.00,170.15,168.74,169.38,19569146,GOOGL,0,0
2024-05-07,168.50,171.76,168.39,171.25,28039696,GOOGL,0,0


<Figure size 1200x800 with 0 Axes>

In [4]:
def stop_loss_calculator(buy_qty, buy_price, stop_loss_factor):

    '''
    This is a calculator to return the exist stop loss price of a trade

    input parameters:
        buy_qty: the total quantity of security bought, INTEGER
        buy_price: the unit share price of the security bought,buy_qty, buy_price, DECIMAL
        stop_loss_factor: the percentage trader is willing to lose at the exist position, DECIMAL
    '''

    total_deployed_cap = buy_qty * buy_price
    stop_loss_price = round(buy_price - ((total_deployed_cap * stop_loss_factor) / buy_qty), 2)
    
    return stop_loss_price



def stop_profit_calculator(buy_qty, buy_price, stop_profit_factor):

    '''
    This is a calculator to return the exist stop profit price of a trade

    input parameters:
        buy_qty: the total quantity of security bought, INTEGER
        buy_price: the unit share price of the security bought, DECIMAL
        stop_profit_factor: the percentage trader is willing to lose at the exist position, DECIMAL
    '''

    total_deployed_cap = buy_qty * buy_price
    stop_profit_price = round((total_deployed_cap * (1+stop_profit_factor)) / buy_qty, 2)

    return stop_profit_price

In [5]:
stop_loss_calculator(buy_qty=18
                     ,buy_price=185.59
                     ,stop_loss_factor=0.03)

180.02